In [1]:
import plwordnet
import spacy

nlp = spacy.load('pl_core_news_sm')
wn = plwordnet.load('../data/plwordnet_4_2.xml')

def generalize_text(text: str) -> str:
    lemmatized_text = []

    doc = nlp(text=text)
    for token in doc:
        lemmatized_text.append(token.lemma_.lower())

    generalized_text = []
    for word in lemmatized_text:

        try:
            if len(word <= 3):
                generalized_text.append(word)
            else:
                lu = wn.find(word+'.2')
                if len(wn.hypernyms(lu.synset)) < 1:
                    generalized_text.append(word)
                else:
                    generalized_text.append(str(wn.hypernyms(lu.synset)[0])[1:-1])
        except Exception as e:
            generalized_text.append(word)

    return ' '.join(generalized_text)

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer

def delete_hashs(df: pd.DataFrame) -> pd.DataFrame:
    df = df[~df['text'].astype(str).str.startswith('#')]
    df = df.reset_index(drop=True)
    return df

def encode_labels(df: pd.DataFrame) -> pd.DataFrame:
    for col in ['Joy', 'Trust', 'Anticipation', 'Surprise', 'Fear','Sadness', 'Disgust', 'Anger', 'Positive', 'Negative', 'Neutral']:
        df[col] = df[col].apply(lambda x: 1 if x else 0)
    return df

def generalize(df: pd.DataFrame) -> pd.DataFrame:
    for index in df.index.to_list():
        df.loc[index, 'text'] = generalize_text(str(df.loc[index, 'text']))
    return df

def embed_text(df: pd.DataFrame) -> pd.DataFrame:
    model = SentenceTransformer('sdadas/st-polish-paraphrase-from-distilroberta')
    
    corpus = [str(df.loc[index, 'text']) for index in df.index.to_list()]
    embeddings = model.encode(corpus)

    embedding_column_names = [f'embedding_{i}' for i in range(embeddings.shape[1])]
    
    df_embeddings = pd.DataFrame(embeddings, columns=embedding_column_names)
    df = pd.concat([df_embeddings, df], axis=1)
    return df

def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.str.lower()
    return df

def transform_texts(df: pd.DataFrame) -> pd.DataFrame:    
    result_df = delete_hashs(df=df)
    result_df = encode_labels(df=result_df)
    result_df = generalize(df=result_df)
    result_df = embed_text(df=result_df)
    result_df = rename_columns(df=result_df)
    return result_df

def delete_empty(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['text'].astype(str).str.len() > 2]
    df = df.reset_index(drop=True)
    return df

def transform_sentences(df: pd.DataFrame) -> pd.DataFrame:
    result_df = pd.DataFrame(data={
        'text': [],
        'Joy': [], 'Trust': [], 'Anticipation': [], 'Surprise': [], 'Fear': [], 'Sadness': [],
           'Disgust': [], 'Anger': [], 'Positive': [], 'Negative': [], 'Neutral': []
        }
    )
    
    sentences = []
    
    for index in df.index.tolist():
        if (str)(df.loc[index, 'text']).startswith('#'):
            sentence = " ".join(sentences)
            df.loc[index, 'text'] = sentence
            result_df = pd.concat([result_df, df.loc[[index]]])
            sentences = []
        else:
            sentences.append((str)(df.loc[index, 'text']))
            
    result_df = delete_hashs(df=result_df)
    result_df = delete_empty(df=result_df)
    result_df = encode_labels(df=result_df)
    result_df = generalize(df=result_df)
    result_df = embed_text(df=result_df)
    result_df = rename_columns(df=result_df)
    return result_df

In [3]:
import os
from typing import List

def load_data() -> List:
    data = []
    
    for name in ['train', 'val', 'test']:
        for category in ['texts', 'sentences']:
            if os.path.exists(f'../data/clean/nn_{name}_{category}.csv'):
                df = pd.read_csv(f'../data/clean/nn_{name}_{category}.csv', index_col=0)
            else:
                df = pd.read_csv(f'../data/raw/{name}.csv')
                if category == 'texts':
                    df = transform_texts(df=df)
                elif category == 'sentences':
                    df = transform_sentences(df=df)
                df.to_csv(f'../data/clean/nn_{name}_{category}.csv')

            data.append(df)        
    return data

In [4]:
data = load_data()
train_texts = data[0]
train_sentences = data[1]
val_texts = data[2]
val_sentences = data[3]
test_texts = data[4]
test_sentences = data[5]

In [5]:
train_texts.head(10)

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,trust,anticipation,surprise,fear,sadness,disgust,anger,positive,negative,neutral
0,0.562764,-0.397882,-0.267292,-0.567739,-1.149851,0.662654,0.128688,0.475320,-0.341749,0.943553,...,0,0,1,0,1,0,1,0,1,0
1,-0.025572,-0.235864,-0.290113,-0.991783,-0.482610,-0.079471,-0.227736,-1.217292,0.438888,-0.221104,...,0,0,0,0,1,1,1,0,1,0
2,-0.475408,0.156776,-0.210054,-1.350016,-0.126225,-0.365511,0.108856,-0.143321,0.438478,0.077761,...,0,0,0,0,0,1,1,0,1,0
3,-0.336756,0.084884,-0.059477,-0.480968,0.186055,-1.020058,-0.095881,0.175767,-0.000036,0.103957,...,0,0,0,0,1,1,1,0,1,0
4,-0.477066,-0.384985,0.092087,-0.471120,-0.756108,-0.020734,-0.597373,-0.533785,0.817650,-0.484433,...,0,0,0,0,0,0,1,0,1,1
5,-0.496382,-0.243892,-0.093434,-0.223938,0.163552,-0.107771,-0.155038,-0.587985,-0.201445,0.310824,...,0,0,0,0,0,0,0,1,0,1
6,0.118517,0.140505,-0.107383,-0.213710,-0.135078,-0.422372,0.039289,-0.559245,-0.158402,-0.107544,...,0,0,0,0,0,0,0,1,0,1
7,0.097196,-0.078515,-0.249001,-0.379476,-0.063541,-0.161061,-0.200802,-0.974016,-0.269598,-0.232394,...,0,0,1,0,1,0,0,0,1,0
8,0.251442,0.039844,-0.041393,0.003911,0.021001,-0.140833,0.306735,0.057826,-0.162801,-0.397339,...,0,0,0,0,0,0,1,1,1,1
9,0.270282,0.079601,0.281395,-0.319867,0.225556,-0.520233,-0.212180,-0.398634,0.146555,-0.211571,...,0,0,0,0,0,0,1,0,1,0


In [6]:
train_sentences.head(10)

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,trust,anticipation,surprise,fear,sadness,disgust,anger,positive,negative,neutral
0,-0.107992,-0.544149,0.053766,-0.632760,-0.311535,-0.581503,-0.313330,-0.348168,0.349289,-0.406877,...,0,0,1,0,1,1,1,0,1,0
1,0.311132,-0.028671,0.441330,0.269969,-0.503570,-0.387672,0.084179,0.129839,0.064349,-0.002668,...,0,0,0,0,0,0,0,0,0,1
2,-0.059933,-0.061721,0.278209,-0.076590,-0.328901,-0.568747,0.589950,-0.501475,0.192955,-0.235708,...,0,0,0,0,0,0,0,1,0,1
3,-0.768429,0.329751,-0.046399,0.045369,-0.539526,-0.464502,0.219784,-0.005870,0.543218,0.019147,...,0,0,0,0,1,0,0,1,1,0
4,-0.079323,-0.283436,-0.049123,0.551994,-0.150095,-0.236103,-0.445485,0.063526,-0.037673,0.159559,...,1,0,0,0,0,0,0,1,0,0
5,0.008165,-0.044767,-0.019482,-0.543493,0.151010,-0.438446,-0.115624,-0.558962,-0.040636,-0.198771,...,0,0,0,0,1,0,1,0,1,0
6,0.192084,0.341302,0.191277,-0.035575,-0.619834,-0.517675,0.101253,-0.099043,-0.411317,-0.308393,...,0,1,0,0,0,0,0,1,0,0
7,-0.034829,-0.051287,0.388698,0.251668,-0.303737,-0.325470,-0.421632,0.267487,-0.006021,-0.086745,...,0,0,0,0,1,0,0,0,1,0
8,-0.085704,0.031369,0.434524,-0.142889,-0.652816,-0.535798,0.209822,-0.081895,-0.156835,-0.037204,...,1,1,0,0,0,0,0,1,0,1
9,0.134724,0.155525,0.143574,0.011912,-0.135043,-0.729203,-0.419711,-0.199815,-0.166667,0.060348,...,0,0,0,0,0,0,0,1,0,0


In [7]:
X_train_texts = train_texts.iloc[:, :768] # + train_sentences.iloc[:, :768]
y_train_texts = train_texts.iloc[:, 769:] # + train_sentences.iloc[:, 769:]

X_val_texts = val_texts.iloc[:, :768] # + val_sentences.iloc[:, :768]
y_val_texts = val_texts.iloc[:, 769:] # + val_sentences.iloc[:, 769:]

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

input_dim = X_train_texts.shape[1]
output_dim = y_train_texts.shape[1]

model = Sequential([
    Dense(1024, activation='relu', input_shape=(input_dim, )),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(output_dim, activation='sigmoid')
])

c:\Users\mchoj\Documents\GitHub\emotion-and-sentiment-recognition\.venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
from keras.metrics import Precision, Recall

optimizer = Adam(learning_rate=0.001, decay=1e-6)

model.compile(
    optimizer=optimizer, 
    loss='binary_crossentropy', 
    metrics=[
        'binary_accuracy',
        Precision(thresholds=0.5, name='precision'),
        Recall(thresholds=0.5, name='recall'),
    ]
)

c:\Users\mchoj\Documents\GitHub\emotion-and-sentiment-recognition\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,830,539 (6.98 MB)

 Trainable params: 1,826,699 (6.97 MB)

 Non-trainable params: 3,840 (15.00 KB)

In [11]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

history = model.fit(
    X_train_texts, y_train_texts,
    epochs=16,
    batch_size=8,
    validation_data=(X_val_texts, y_val_texts),
    callbacks=[early_stop, reduce_lr],
)

Epoch 1/16
800/800 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - binary_accuracy: 0.7125 - loss: 0.5749 - precision: 0.4917 - recall: 0.6042 - val_binary_accuracy: 0.8488 - val_loss: 0.3521 - val_precision: 0.7836 - val_recall: 0.6341 - learning_rate: 0.0010
Epoch 2/16
800/800 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - binary_accuracy: 0.8316 - loss: 0.3866 - precision: 0.7310 - recall: 0.5951 - val_binary_accuracy: 0.8527 - val_loss: 0.3444 - val_precision: 0.7917 - val_recall: 0.6414 - learning_rate: 0.0010
Epoch 3/16
800/800 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - binary_accuracy: 0.8475 - loss: 0.3618 - precision: 0.7630 - recall: 0.6247 - val_binary_accuracy: 0.8603 - val_loss: 0.3321 - val_precision: 0.7930 - val_recall: 0.6768 - learning_rate: 0.0010
Epoch 4/16
800/800 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - binary_accuracy: 0.8527 - loss: 0.3535 - precision: 0.7749 - recall: 0.6328 - val_binary_accuracy: 0.8628 - val_loss: 0.3312 - val_precision: 0.7983 - val_recall: 0.6813 - learning_rate: 0.0010


In [12]:
model.save('../models/neural_network.keras') 

In [13]:
import keras

history = keras.models.load_model('../models/neural_network.keras')

In [14]:
import numpy as np
from typing import Dict

def evaluate_texts(model, X_test: pd.DataFrame, y_test: pd.DataFrame, threshold: float=0.5) -> Dict:
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > threshold).astype(int)
    
    y_test = y_test.values
    
    labels = ['Joy', 'Trust', 'Anticipation', 'Surprise', 'Fear', 'Sadness', 'Disgust', 'Anger', 'Positive', 'Negative', 'Neutral']
    metrics = {}
    
    metrics['F1-score macro texts'] = 0
    for label in labels:
        metrics[f"Precision {label}"] = 0
        metrics[f"Recall {label}"] = 0
        metrics[f"F1-score {label}"] = 0
        metrics[f"TP {label}"] = 0
        metrics[f"FP {label}"] = 0
        metrics[f"TN {label}"] = 0
        metrics[f"FN {label}"] = 0
    
    for i, label in enumerate(labels):
        metrics[f"TP {label}"] = int(np.sum((y_pred_binary[:, i] == 1) & (y_test[:, i] == 1)))
        metrics[f"FP {label}"] = int(np.sum((y_pred_binary[:, i] == 1) & (y_test[:, i] == 0)))
        metrics[f"TN {label}"] = int(np.sum((y_pred_binary[:, i] == 0) & (y_test[:, i] == 0)))
        metrics[f"FN {label}"] = int(np.sum((y_pred_binary[:, i] == 0) & (y_test[:, i] == 1)))
        
    for label in labels:
        metrics[f"Precision {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FP {label}"] + 1e-8)
        metrics[f"Recall {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FN {label}"] + 1e-8)
        metrics[f"F1-score {label}"] = 2 * (metrics[f"Precision {label}"] * metrics[f"Recall {label}"]) / (metrics[f"Precision {label}"] + metrics[f"Recall {label}"] + 1e-8)

    metrics['F1-score macro texts'] = sum(metrics[f"F1-score {label}"] for label in labels) / len(labels)
    
    metrics = {k: v for (k, v) in metrics.items() if ('Precision' in k) or ('Recall' in k) or ('F1' in k)}

    return metrics

In [15]:
def evaluate_sentences_v1(model, X_test: pd.DataFrame, y_test: pd.DataFrame, threshold: float=0.5) -> Dict:
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > threshold).astype(int)
    
    y_test = y_test.values
    
    labels = ['Joy', 'Trust', 'Anticipation', 'Surprise', 'Fear', 'Sadness', 'Disgust', 'Anger', 'Positive', 'Negative', 'Neutral']
    metrics = {}
    
    metrics['F1-score macro sentences'] = 0
    for label in labels:
        metrics[f"Precision {label}"] = 0
        metrics[f"Recall {label}"] = 0
        metrics[f"F1-score {label}"] = 0
        metrics[f"TP {label}"] = 0
        metrics[f"FP {label}"] = 0
        metrics[f"TN {label}"] = 0
        metrics[f"FN {label}"] = 0
    
    for i, label in enumerate(labels):
        metrics[f"TP {label}"] = int(np.sum((y_pred_binary[:, i] == 1) & (y_test[:, i] == 1)))
        metrics[f"FP {label}"] = int(np.sum((y_pred_binary[:, i] == 1) & (y_test[:, i] == 0)))
        metrics[f"TN {label}"] = int(np.sum((y_pred_binary[:, i] == 0) & (y_test[:, i] == 0)))
        metrics[f"FN {label}"] = int(np.sum((y_pred_binary[:, i] == 0) & (y_test[:, i] == 1)))
        
    for label in labels:
        metrics[f"Precision {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FP {label}"] + 1e-8)
        metrics[f"Recall {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FN {label}"] + 1e-8)
        metrics[f"F1-score {label}"] = 2 * (metrics[f"Precision {label}"] * metrics[f"Recall {label}"]) / (metrics[f"Precision {label}"] + metrics[f"Recall {label}"] + 1e-8)

    metrics['F1-score macro sentences'] = sum(metrics[f"F1-score {label}"] for label in labels) / len(labels)

    metrics = {k: v for (k, v) in metrics.items() if ('Precision' in k) or ('Recall' in k) or ('F1' in k)}

    return metrics


In [16]:
def get_hash_indeces(df: pd.DataFrame) -> List:
    hash_indices = []
    for index in df.index.tolist():
        if str(df.loc[index, 'text']).startswith('#'):
            hash_indices.append(index)
          
    result_hash_indices = []   
    i = 0
    for index in hash_indices:
        index = index - i
        result_hash_indices.append(index)
        i += 1
    
    return result_hash_indices

In [17]:
def evaluate_sentences_v2(model: keras.Model, X_test: pd.DataFrame, y_test: pd.DataFrame, threshold: float=1): # -> Dict:
    raw_data = encode_labels(pd.read_csv('../data/raw/test.csv'))
    hash_indices = get_hash_indeces(raw_data)
    
    y_pred = model.predict(X_test)
    y_test = y_test.values
    
    labels = ['Joy', 'Trust', 'Anticipation', 'Surprise', 'Fear', 'Sadness', 'Disgust', 'Anger', 'Positive', 'Negative', 'Neutral']
    metrics = {}
    
    metrics['F1-score macro sentences'] = 0
    for label in labels:
        metrics[f"Precision {label}"] = 0
        metrics[f"Recall {label}"] = 0
        metrics[f"F1-score {label}"] = 0
        metrics[f"TP {label}"] = 0
        metrics[f"FP {label}"] = 0
        metrics[f"TN {label}"] = 0
        metrics[f"FN {label}"] = 0
    
    y_true_segments = []
    y_pred_segments = []
    
    i = 0
    start = 0
    for index in hash_indices:
        pred_sum = np.zeros(11, dtype=np.float64)
        for y_pred_i in y_pred[start:index]:
            pred_sum += y_pred_i
            
        y_true_segments.append(raw_data.iloc[index+0, 1:].to_numpy())
        y_pred_segments.append((pred_sum >= threshold).astype(int))
                    
        i += 1
        start = index

    y_true_segments = np.array(y_true_segments)
    y_pred_segments = np.array(y_pred_segments)

    for i, label in enumerate(labels):
        metrics[f"TP {label}"] = int(np.sum((y_pred_segments[:, i] == 1) & (y_true_segments[:, i] == 1)))
        metrics[f"FP {label}"] = int(np.sum((y_pred_segments[:, i] == 1) & (y_true_segments[:, i] == 0)))
        metrics[f"TN {label}"] = int(np.sum((y_pred_segments[:, i] == 0) & (y_true_segments[:, i] == 0)))
        metrics[f"FN {label}"] = int(np.sum((y_pred_segments[:, i] == 0) & (y_true_segments[:, i] == 1)))
        
    for label in labels:
        metrics[f"Precision {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FP {label}"] + 1e-8)
        metrics[f"Recall {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FN {label}"] + 1e-8)
        metrics[f"F1-score {label}"] = 2 * (metrics[f"Precision {label}"] * metrics[f"Recall {label}"]) / (metrics[f"Precision {label}"] + metrics[f"Recall {label}"] + 1e-8)

    metrics['F1-score macro sentences'] = sum(metrics[f"F1-score {label}"] for label in labels) / len(labels)

    metrics = {k: v for (k, v) in metrics.items() if ('Precision' in k) or ('Recall' in k) or ('F1' in k)}

    return metrics

In [18]:
texts_metrics = evaluate_texts(history, X_test=test_texts.iloc[:, :768], y_test=test_texts.iloc[:, 769:], threshold=0.5)
print(texts_metrics)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
{'F1-score macro texts': 0.570798261912883, 'Precision Joy': 0.7992766726799407, 'Recall Joy': 0.8036363636217521, 'F1-score Joy': 0.8014505842874088, 'Precision Trust': 0.6301369862725965, 'Recall Trust': 0.6602870813081202, 'F1-score Trust': 0.6448598080567081, 'Precision Anticipation': 0.5303030302226814, 'Recall Anticipation': 0.2799999999776, 'F1-score Anticipation': 0.3664921420355803, 'Precision Surprise': 0.0, 'Recall Surprise': 0.0, 'F1-score Surprise': 0.0, 'Precision Fear': 0.6999999992999999, 'Recall Fear': 0.11864406777650101, 'F1-score Fear': 0.20289854818735562, 'Precision Sadness': 0.8259385665388065, 'Recall Sadness': 0.879999999984, 'F1-score Sadness': 0.8521126710463574, 'Precision Disgust': 0.6249999999660326, 'Recall Disgust': 0.473251028787109, 'F1-score Disgust': 0.5386416812529001, 'Precision Anger': 0.6739130434294266, 'Recall Anger': 0.44285714283605443, 'F1-score Anger': 0.5344827538040032, 'Precision Positive': 0.824212

In [19]:
sentences_metrics_v1 = evaluate_sentences_v1(history, X_test=test_sentences.iloc[:, :768], y_test=test_sentences.iloc[:, 769:], threshold=0.5)
print(sentences_metrics_v1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
{'F1-score macro sentences': 0.621044168696023, 'Precision Joy': 0.8554216866439251, 'Recall Joy': 0.8255813952528395, 'F1-score Joy': 0.840236681292672, 'Precision Trust': 0.6363636361707988, 'Recall Trust': 0.6999999997666666, 'F1-score Trust': 0.6666666614663644, 'Precision Anticipation': 0.7142857132653061, 'Recall Anticipation': 0.41666666631944443, 'F1-score Anticipation': 0.5263157842659281, 'Precision Surprise': 0.0, 'Recall Surprise': 0.0, 'F1-score Surprise': 0.0, 'Precision Fear': 0.3333333322222222, 'Recall Fear': 0.08333333326388888, 'F1-score Fear': 0.1333333299555556, 'Precision Sadness': 0.8947368420110804, 'Recall Sadness': 0.9340659339632895, 'F1-score Sadness': 0.9139784895276911, 'Precision Disgust': 0.6486486484733381, 'Recall Disgust': 0.5333333332148148, 'F1-score Disgust': 0.5853658485633552, 'Precision Anger': 0.8787878785215794, 'Recall Anger': 0.6444444443012346, 'F1-score Anger': 0.7435897385174228, 'Precision Positive':

In [20]:
sentences_metrics_v2 = evaluate_sentences_v2(history, X_test=test_sentences.iloc[:, :768], y_test=test_sentences.iloc[:, 769:], threshold=0.5)
print(sentences_metrics_v2)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
{'F1-score macro sentences': 0.1677332432885955, 'Precision Joy': 0.4999999998076923, 'Recall Joy': 0.18309859152350727, 'F1-score Joy': 0.2680412331342332, 'Precision Trust': 0.20833333324652778, 'Recall Trust': 0.19999999992, 'F1-score Trust': 0.2040816275718452, 'Precision Anticipation': 0.049999999975, 'Recall Anticipation': 0.0624999999609375, 'F1-score Anticipation': 0.05555555058642019, 'Precision Surprise': 0.13333333324444444, 'Recall Surprise': 0.1999999998, 'F1-score Surprise': 0.15999999507200013, 'Precision Fear': 0.0, 'Recall Fear': 0.0, 'F1-score Fear': 0.0, 'Precision Sadness': 0.5172413791319858, 'Recall Sadness': 0.19230769228303748, 'F1-score Sadness': 0.2803738277718579, 'Precision Disgust': 0.13043478255198487, 'Recall Disgust': 0.09999999996666667, 'F1-score Disgust': 0.11320754221431137, 'Precision Anger': 0.13043478255198487, 'Recall Anger': 0.0909090908815427, 'F1-score Anger': 0.10714285226403084, 'Precision Positive': 0.5

In [21]:
def calculate_final_score(text_metrics: Dict, sentences_metrics: Dict) -> float:
    return (text_metrics['F1-score macro texts'] + sentences_metrics['F1-score macro sentences']) / 2

In [22]:
calculate_final_score(texts_metrics, sentences_metrics_v1)

0.595921215304453

In [23]:
calculate_final_score(texts_metrics, sentences_metrics_v2)

0.36926575260073924